# Fit the sinusoidal amplitudes

First I have to create a PDF for fitting. We will do this in a seperate notebook. [seperate notebook](CreatePDF.ipynb). This creates a c++ class which returns the value of the PDF. As such it can be as general a function as you like.

First load the fit classes.

In [ ]:
import ROOT
ROOT.gROOT.ProcessLine(".x $BRUFIT/macros/LoadBru.C")
%jsroot

Now load and compile the PhiAsymmetry PDF class you created.

In [ ]:
ROOT.Loader.Compile("PhiAsymmetry.cxx");

Use an instance of the FitManager class to organise the fit and give it an ouput directory.

In [ ]:
fm = ROOT.FitManager()
fm.SetUp().SetOutDir("outObs1/");

Now load the experimentally measured variables (or observables). I must use the name of the variables in the tree.

Note the slighlty different notation for the Category PolState. If any events had a PolState of 0 for example, then they would not be included in the fit.

In [ ]:
fm.SetUp().LoadVariable("Phi[-180,180]"); 
fm.SetUp().LoadVariable("Pol[0,1]"); 
fm.SetUp().LoadCategory("PolState[Polp=1,Polm=-1]");

If I want to use any other variable, or example to apply a cut, I must load it as an AuxVar. The limits given will be applied to the datset. Here we will not apply it, but it is left commented out to show what you could do.

In [ ]:
#fm.SetUp().LoadAuxVar("M1[0,10]"); //Load Aux Var, limits used as cut 
#fm.SetUp().AddCut("M1>2"); //Additional cut based on vars or aux vars

IMPORTANT here we set the event ID variable. Each event in your tree should have a unique id with which to synchronise with weights. This is useful as it allows us to break the data up, perform seperate fits then combine the weights afterwards. The weights are not written into the input tree, rather they are stored seperately (as HS::Weights) and combined when required. This allows you to use various sets of weights when performing fits or regenerate the weights after some corrections etc.

In [ ]:
fm.SetUp().SetIDBranchName("fgID");

Create and load into the fit manager my PDF class. The class is callled PhiAsymmetry, this object is given the name SigAsym, which can be used to assign simulated data later. Here I explicitly declare my fit parameters A and B with there initial values and ranges. Phi, Pol and PolState all match the variables loaded above.

In [ ]:
fm.SetUp().FactoryPDF("PhiAsymmetry::SigAsym( Phi,Pol,PolState,A[0,-1,1],B[0,-1,1] )");
fm.SetUp().LoadSpeciesPDF("SigAsym",1);

Now I load my data, here I will use both data to fit and MC data for the integral calculation. Note in the latter case the string "SigAsym" matches the name given to the PDF object above. In principal I could fit multiple PDFs and give them each different MC data.

The strings are treename, filename and PDF name (simulation only).

In [ ]:
fm.LoadData("MyModel","Data.root");
fm.LoadSimulated("MyModel","MC.root","SigAsym");

Now I attach the weights from my sPlot fit. I want to use the signal weights which were given the name "Signal" in the sPlot notebook. 

Also there are two possible sets of weights from using a Gaussian or simulated PDF. You can switch between either here.

In [ ]:
fm.Data().LoadWeights("Signal","outSplot/Tweights.root");
#fm.Data().LoadWeights("Signal","outSplotSim/Tweights.root"); #use weights from simulated PDF

Now run the fits. I use the Process classes which allow me to choose between running directly here on a single core or multicore via PROOF-lite. It doesn't make sense to run with PROOF unless multiple splits have been defined with LoadBinVar or you are using Bootstrap, in which case you should relate the number of cores requested to the number of splits or bootstraps

In [ ]:
ROOT.Here.Go(fm)

All plots and fit parameters will be saved into the output directory Results.root file. If LoadBinVar splits were applied then they will be in directories related to the bin name.

If the plot does not appear you can try removing the %jsroot in the first cell. Unfortunately this will mean the histograms will not be interactive.